In [148]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Загрузка данных

In [149]:
%%capture
!wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx

"wget" �� ���� ����७��� ��� ���譥�
��������, �ᯮ��塞�� �ணࠬ��� ��� ������ 䠩���.


In [150]:
data = pd.read_excel('data_ford_price.xlsx') 

## Предобработка данных

In [151]:
data = data[['price','year', 'cylinders', 'odometer', 'lat', 'long', 'weather']]
data.dropna(inplace = True)

In [152]:
data.head()

,price,year,cylinders,odometer,lat,long,weather
0,43900,2016,6,43500,36.471500,-82.483400,59.0
1,15490,2009,8,98131,40.468826,-74.281734,52.0
2,2495,2002,8,201803,42.477134,-82.949564,45.0
3,1300,2000,8,170305,40.764373,-82.349503,49.0
5,6995,2003,8,167662,45.518031,-122.578752,50.0


#  Отбор наиболее полезных признаков двумя методами:

## Метод рекурсивного исключения признаков

In [153]:
from sklearn.feature_selection import RFE

In [154]:
y = data['price']
x = data.drop(columns='price')

In [155]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=40)

In [156]:
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'lat'], dtype=object)

In [157]:
# Сделаем новые тренеровочный и тестовый датафреймы X_train_RFE, X_test_RFE только с найденными методом RFE признаками 
cols_RFE = selector.get_support(indices=True)
X_train_RFE = X_train.iloc[:,cols_RFE]
X_test_RFE = X_test.iloc[:,cols_RFE]
X_train_RFE.head()

,year,cylinders,lat
4641,2011,6,31.316064
3188,2010,8,42.892121
135,2008,8,35.887189
962,2011,8,43.781240
6188,2006,8,42.484503


##  МЕТОДЫ ВЫБОРА ПРИЗНАКОВ НА ОСНОВЕ ФИЛЬТРОВ

In [158]:
from sklearn.feature_selection import SelectKBest, f_regression, f_classif

In [159]:
selector = SelectKBest(f_regression, k=3)
selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'cylinders', 'odometer'], dtype=object)

In [160]:
# Сделаем новые тренеровочные датафрейы X_train_KBest X_test_KBest только с найденными методом KBest признаками 
cols_KBest = selector.get_support(indices=True)
X_train_KBest = X_train.iloc[:,cols_KBest]
X_test_KBest= X_test.iloc[:,cols_KBest]
X_train_KBest.head()

,year,cylinders,odometer
4641,2011,6,122882
3188,2010,8,144570
135,2008,8,136124
962,2011,8,136668
6188,2006,8,187000


# Обучение модели на признаках, найденных методом RFE

In [161]:
## Обучение модели
model = LinearRegression()
model.fit(X_train_RFE, y_train)

y_predicted = model.predict(X_test_RFE)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)

MAE: 5096.570


# Обучение модели на признаках, найденных методом SelectKBest

In [162]:
## Обучение модели
model = LinearRegression()
model.fit(X_train_KBest, y_train)

y_predicted = model.predict(X_test_KBest)
 
mae = mean_absolute_error(y_test, y_predicted)
print('MAE: %.3f' % mae)


MAE: 4708.946


Вопрос: «Какой метод отбора признаков показал наилучший результат на тестовой выборке?» 
Отбор трех трех наиболее полезных принаков методом SelectKBest показал лучший лезультат по метрике MAE. 
MAE 4709 против 5096 полученной при выборе трех признаков методом RFE